<a href="https://colab.research.google.com/github/Justy-11/deeplearning_fundamentals/blob/main/keras_tuner_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install keras-tuner --upgrade

In [3]:
import keras_tuner
from tensorflow import keras

Using TensorFlow backend


In [4]:
df = pd.read_csv('/content/diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# correlation between Outcome and others
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
# [:,:-1]: This part of the code specifies the row and column selection. Here, : is used for selecting all rows, and :-1 is used for selecting all columns except the last one.
# In other words, it selects all rows and all columns up to, but not including, the last column in the DataFrame.
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [8]:
X.shape, y.shape

((768, 8), (768,))

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
# scale from -1 to 1
X = scaler.fit_transform(X)

In [11]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42)

In [13]:
X_train, y_train

(array([[ 1.82781311e+00, -1.43636781e+00,  6.66618252e-01, ...,
          1.65936998e-01, -6.30831221e-01,  4.04942367e-01],
        [ 9.36913723e-01,  5.04055196e-01,  1.08020025e+00, ...,
          9.41978774e-04, -2.44256030e-01,  4.90030120e-01],
        [-2.50952128e-01,  1.16129525e+00, -2.63941247e-01, ...,
         -1.00593418e-01, -5.34187424e-01, -7.86286177e-01],
        ...,
        [ 1.82781311e+00, -6.22642036e-01,  8.73409251e-01, ...,
          1.72704372e+00,  2.00573238e+00,  4.04942367e-01],
        [-1.14185152e+00,  6.29243777e-01, -3.57259724e+00, ...,
          1.32090213e+00, -8.05998104e-01, -3.60847411e-01],
        [-1.14185152e+00,  1.28489452e-01,  1.39038675e+00, ...,
         -1.20479085e+00, -6.33851340e-01, -1.04154944e+00]]),
 array([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       

In [14]:
import keras_tuner as kt

In [15]:
# hp argument to define the hyperparameters during model creation.
def build_model(hp):
  model = keras.Sequential()

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

  model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32]),
      activation='relu'))
  model.add(keras.layers.Dense(1, activation='relu'))
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
  return model

In [16]:
# objective='val_accuracy': This specifies the objective to optimize during the hyperparameter search. In this case, the objective is to maximize the validation accuracy of the model.
# The hyperparameter tuning process will try different hyperparameter combinations to find the configuration that produces the best validation accuracy.
# max_trials=5: This sets the maximum number of trials (i.e., hyperparameter combinations) to be tested during the random search. In this example, the hyperparameter search will
# run a maximum of 5 trials, each with a different set of hyperparameters.
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5)

In [17]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.46991869807243347

Best val_accuracy So Far: 0.6975609660148621
Total elapsed time: 00h 00m 16s


In [18]:
best_model = tuner.get_best_models()[0]

In [19]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
optimizer: rmsprop
units: 32
Score: 0.6975609660148621

Trial 0 summary
Hyperparameters:
optimizer: adadelta
units: 32
Score: 0.6813008189201355

Trial 2 summary
Hyperparameters:
optimizer: adam
units: 8
Score: 0.4878048896789551

Trial 4 summary
Hyperparameters:
optimizer: adam
units: 32
Score: 0.46991869807243347

Trial 3 summary
Hyperparameters:
optimizer: sgd
units: 16
Score: 0.3333333432674408


In [20]:
# !rm -r /content/untitled_project